In [9]:
import vanna as vn
from vanna.remote import VannaDefault
vn = VannaDefault(model='elie', api_key='02d3fb738a4c42de8b2ca3ec356450c0')
# Remplacez par vos informations de connexion
import os
from dotenv import load_dotenv

load_dotenv()

odbc_conn_str = f"""DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={os.getenv('DB_SERVER')};DATABASE={os.getenv('DB_DATABASE')};UID={os.getenv('DB_USER')};PWD={os.getenv('DB_PASSWORD')};TrustServerCertificate={os.getenv('DB_TRUST_SERVER_CERTIFICATE')}"""

vn.connect_to_mssql(odbc_conn_str=odbc_conn_str)


In [10]:
response = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")
print(response)  # Afficher la réponse brute
print(type(response))  # Pour voir si c'est bien un objet contenant du JSON



       TABLE_CATALOG TABLE_SCHEMA              TABLE_NAME         COLUMN_NAME  \
0     3E_80_Commande          dbo  Mahn_Mahnbestandposten                  ID   
1     3E_80_Commande          dbo  Mahn_Mahnbestandposten          MahnlaufID   
2     3E_80_Commande          dbo  Mahn_Mahnbestandposten       MahnbestandID   
3     3E_80_Commande          dbo  Mahn_Mahnbestandposten          Mahnsperre   
4     3E_80_Commande          dbo  Mahn_Mahnbestandposten  MahnDatumBisSperre   
...              ...          ...                     ...                 ...   
5832  3E_80_Commande          dbo            D_Produktion             Element   
5833  3E_80_Commande          dbo            D_Produktion             Fluegel   
5834  3E_80_Commande          dbo            D_Produktion              Nummer   
5835  3E_80_Commande          dbo            D_Produktion               Wagen   
5836  3E_80_Commande          dbo            D_Produktion                Fach   

      ORDINAL_POSITION COLU

In [11]:
# Récupérer les métadonnées du schéma
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME IN ('P_Zubeh', 'A_Kopf', 'P_Kaufm', 'A_Vorgang', 'P_Teil')")



# Générer le plan d'entraînement
plan = vn.get_training_plan_generic(df_information_schema)
print(plan)

# Entraîner le modèle
vn.train(plan=plan)

vn.train(
    question="liste moi les 10 dernieres commandes ?", 
    sql="""SELECT TOP 10 * 
        FROM A_kopf
        INNER JOIN dbo.A_Vorgang AS DateSerie ON DateSerie.ID_A_Kopf = Paramgen.ID_A_Kopf
        WHERE DateSerie.Nummer = '_Cde en série'
        ORDER BY DateSerie.Datum DESC"""
)


Train on Information Schema: 3E_80_Commande.dbo A_Kopf
Train on Information Schema: 3E_80_Commande.dbo A_Vorgang
Train on Information Schema: 3E_80_Commande.dbo P_Kaufm
Train on Information Schema: 3E_80_Commande.dbo P_Teil
Train on Information Schema: 3E_80_Commande.dbo P_Zubeh


'231059eb7471f2eb0bfde438d3913cb8-sql'

In [12]:
print(dir(vn))



['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_api_key', '_dataclass_to_dict', '_endpoint', '_extract_python_code', '_get_databases', '_get_information_schema_tables', '_graphql_endpoint', '_graphql_headers', '_model', '_response_language', '_rpc_call', '_sanitize_plotly_code', 'add_ddl', 'add_ddl_to_prompt', 'add_documentation', 'add_documentation_to_prompt', 'add_question_sql', 'add_sql_to_prompt', 'ask', 'assistant_message', 'config', 'connect_to_bigquery', 'connect_to_clickhouse', 'connect_to_duckdb', 'connect_to_hive', 'connect_to_mssql', 'connect_to_mysql', 'connect_to_oracle', 

In [13]:
reponse = vn.ask("liste moi les 10 dernieres commandes ?")
print(reponse)


SQL Prompt: [{'role': 'system', 'content': "You are a T-SQL / Microsoft SQL Server expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Additional Context \n\nThe following columns are in the Vue_Facture_Jour table in the 3E_80_Commande database:\n\n|     | TABLE_CATALOG   | TABLE_SCHEMA   | TABLE_NAME       | COLUMN_NAME        | DATA_TYPE   |\n|----:|:----------------|:---------------|:-----------------|:-------------------|:------------|\n| 619 | 3E_80_Commande  | dbo            | Vue_Facture_Jour | N° Facture         | int         |\n| 620 | 3E_80_Commande  | dbo            | Vue_Facture_Jour | Type Affaire       | smallint    |\n| 621 | 3E_80_Commande  | dbo            | Vue_Facture_Jour | Type Facture       | smallint    |\n| 622 | 3E_80_Commande  | dbo            | Vue_Facture_Jour | Date Facture       | datetime    |\n| 623 | 3E_80_Commande  | 

In [14]:
from vanna.flask import VannaFlaskApp
VannaFlaskApp(vn).run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on
None
None
None
None
None
None
